In [1]:
from ddpg.DDPG import DDPG
from ddpg.portfolio import PortfolioEnv
from ddpg.StockActor import StockActor
from ddpg.StockCritic import StockCritic
from ddpg.ornstein_uhlenbeck import OrnsteinUhlenbeckActionNoise

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import json 

import os
import sys
fileDir = os.getcwd()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
with open(fileDir+'/config/cnn.json', 'r') as f:
    config = json.load(f)

dataset_path = fileDir + '/data/data1130'
window_size = config["window_length"]
steps = config["steps"]
trading_cost = config["trading_cost"]
start_idx = config["start_idx"]

env = PortfolioEnv(dataset_path,
                   window_size=window_size,
                   steps=steps,
                   trading_cost=trading_cost,
                   start_idx = start_idx)

info = env.reset()
asset_dim = info["observation"].shape[0]
feature_dim = info["observation"].shape[2]

actor_learning_rate = config["actor_learning_rate"]
critic_learning_rate = config["critic_learning_rate"]
tau = config["tau"]
gamma = config["gamma"]
batch_size = config["batch_size"]
nn = config["nn"]
path = config["path"]

with tf.Session() as sess:
    actor = StockActor(sess, asset_dim, window_size, feature_dim, actor_learning_rate, tau, batch_size, nn)
    num_actor_vars = actor.num_trainable_vars
    critic = StockCritic(sess, asset_dim, window_size, feature_dim, critic_learning_rate, tau, gamma, num_actor_vars, nn)
    actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(asset_dim))

    ddpg = DDPG(env, sess, actor, critic, actor_noise, config, nn, 
                model_save_path = fileDir+ path + "ckpt",
                summary_path = fileDir + path + "logdir", 
                train_info_path = fileDir + path + "train_info_cnn.csv")
    ddpg.train()
ddpg.env.save_info(info_path = fileDir + path + "info_cnn.csv")

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
------------------------------------------------------------------------------------------------
Episode: 0, Reward: 0.0140, Qmax: 0.0021, loss: 0.0027


/opt/anaconda-gpu-2/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


------------------------------------------------------------------------------------------------
Episode: 1, Reward: 0.3758, Qmax: 0.0022, loss: 0.0069
------------------------------------------------------------------------------------------------
Episode: 2, Reward: 0.4121, Qmax: 0.0037, loss: 0.0089
------------------------------------------------------------------------------------------------
Episode: 3, Reward: 0.4350, Qmax: 0.0065, loss: 0.0094
------------------------------------------------------------------------------------------------
Episode: 4, Reward: 0.4149, Qmax: 0.0061, loss: 0.0087
------------------------------------------------------------------------------------------------
Episode: 5, Reward: 0.4235, Qmax: 0.0058, loss: 0.0086
------------------------------------------------------------------------------------------------
Episode: 6, Reward: 0.4230, Qmax: 0.0050, loss: 0.0083
----------------------------------------------------------------------------------------